# 🧹 Limpieza de Datos: Dataset Crohn/IBD

**Objetivo:** Limpiar y normalizar los datos filtrados de usuarios con Crohn/IBD

**Input:** `crohn_filtered.csv` (390,765 registros, 2,046 usuarios)

**Tareas:**
1. Separar valores numéricos vs categóricos en `trackable_value`
2. Normalizar nombres de síntomas (estandarizar)
3. Limpiar tratamientos y dosis
4. Manejar valores missing
5. Preparar datos para feature engineering

**Autor:** Asier Ortiz García  
**Fecha:** Octubre 2025

## 📦 Imports y Configuración

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
%matplotlib inline

print('=' * 80)
print('LIMPIEZA DE DATOS: Dataset Crohn/IBD')
print('=' * 80)

LIMPIEZA DE DATOS: Dataset Crohn/IBD


## 1️⃣ Cargar Datos Filtrados

In [20]:
# Cargar datos filtrados del notebook anterior
df = pd.read_csv('../data/processed/crohn_filtered.csv', parse_dates=['checkin_date'])

print(f'✓ Cargado: {len(df):,} registros')
print(f'  Usuarios: {df["user_id"].nunique():,}')
print(f'  Rango: {df["checkin_date"].min()} → {df["checkin_date"].max()}')
print(f'\nDistribución por tipo:')
print(df['trackable_type'].value_counts())

✓ Cargado: 390,765 registros
  Usuarios: 2,046
  Rango: 2015-05-24 00:00:00 → 2019-12-06 00:00:00

Distribución por tipo:
trackable_type
Symptom      161086
Treatment     67169
Weather       59892
Condition     56185
Food          23175
Tag           23101
HBI             157
Name: count, dtype: int64


## 2️⃣ Análisis de `trackable_value`

In [21]:
print('Top 30 valores de trackable_value:')
print(df['trackable_value'].value_counts().head(30))

# Intentar convertir a numérico
df['value_numeric'] = pd.to_numeric(df['trackable_value'], errors='coerce')

numeric_mask = df['value_numeric'].notna()
print(f'\nValores numéricos: {numeric_mask.sum():,} ({numeric_mask.sum()/len(df)*100:.1f}%)')
print(f'Valores no numéricos: {(~numeric_mask).sum():,}')

# Ver distribución por tipo
print('\nNuméricos por tipo:')
print(df[numeric_mask].groupby('trackable_type').size())

print('\nNo numéricos por tipo:')
print(df[~numeric_mask].groupby('trackable_type').size())

Top 30 valores de trackable_value:
trackable_value
0                                75089
1                                53002
2                                46026
3                                28541
4                                15344
partly-cloudy-day                 3641
rain                              2387
1.0 tablet                        2180
0.0                               2012
30.0 mg                           1726
150.0 mcg                         1548
partly-cloudy-night               1462
1000.0 IU                         1439
0.5 packet                        1408
2 capsules                        1403
1 capsule                         1350
150.0 mg                          1099
2 x 500mg                         1033
20 ml                             1011
0.0001                             993
1.0 drop                           987
clear-day                          962
3.0 pill                           954
1 tsp                              932
5 mg         

## 3️⃣ Limpiar Valores por Tipo

Estrategia:
- **Symptoms**: Convertir a escala 0-4 (severidad)
- **Treatments**: Extraer indicador de toma (yes/no) y dosis si es posible
- **Weather**: Categorías fijas
- **Food**: Binario (consumido o no)
- **HBI**: Numérico directo

In [22]:
# Crear columna limpia para cada tipo
df['value_clean'] = df['trackable_value'].copy()

# SÍNTOMAS: Ya deberían ser 0-4 en su mayoría
symptoms = df[df['trackable_type'] == 'Symptom'].copy()
print('Síntomas - distribución de valores:')
print(symptoms['trackable_value'].value_counts().head(10))

# Verificar si hay síntomas con valores fuera de 0-4
symptoms_numeric = pd.to_numeric(symptoms['trackable_value'], errors='coerce')
out_of_range = symptoms_numeric[(symptoms_numeric < 0) | (symptoms_numeric > 4)]
print(f'\nSíntomas fuera de rango 0-4: {len(out_of_range):,}')
if len(out_of_range) > 0:
    print('Valores únicos fuera de rango:', sorted(out_of_range.unique())[:20])

Síntomas - distribución de valores:
trackable_value
0    57066
1    39019
2    34090
3    20314
4    10597
Name: count, dtype: int64

Síntomas fuera de rango 0-4: 0


In [23]:
# TRATAMIENTOS: Extraer dosis
treatments = df[df['trackable_type'] == 'Treatment'].copy()
print('Tratamientos - ejemplos de valores:')
print(treatments['trackable_value'].value_counts().head(20))

# Función para extraer dosis numérica de tratamientos
def extract_dose(value):
    """
    Extrae la dosis numérica de strings como '1.0 tablet', '30.0 mg', '2 capsules'
    """
    if pd.isna(value):
        return np.nan
    
    # Si ya es numérico, devolver
    try:
        return float(value)
    except:
        pass
    
    # Buscar patrones como "30.0 mg", "1.0 tablet", "2 capsules"
    # Debe tener al menos un dígito (no solo punto)
    match = re.search(r'(\d+\.?\d*|\d*\.\d+)', str(value))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan
    
    return np.nan

# Aplicar extracción de dosis
treatments['dose_extracted'] = treatments['trackable_value'].apply(extract_dose)
print(f'\nDosis extraídas: {treatments["dose_extracted"].notna().sum():,} de {len(treatments):,}')
print(f'Dosis promedio: {treatments["dose_extracted"].mean():.2f}')
print(f'Dosis mediana: {treatments["dose_extracted"].median():.2f}')

Tratamientos - ejemplos de valores:
trackable_value
1.0 tablet                       2180
30.0 mg                          1726
150.0 mcg                        1548
1000.0 IU                        1439
0.5 packet                       1408
2 capsules                       1403
1 capsule                        1350
150.0 mg                         1099
2 x 500mg                        1033
20 ml                            1011
1.0 drop                          987
3.0 pill                          954
1 tsp                             932
5 mg                              883
1.0 dose                          798
3 drops                           751
2 x 100.0 mg                      735
20.0 mg                           675
325 mg (35 mg elemental iron)     673
8 pills                           659
Name: count, dtype: int64

Dosis extraídas: 65,806 de 67,169
Dosis promedio: 133.38
Dosis mediana: 3.00


In [24]:
# WEATHER: Categorías
weather = df[df['trackable_type'] == 'Weather'].copy()
print('Weather - categorías:')
print(weather['trackable_value'].value_counts())

Weather - categorías:
trackable_value
partly-cloudy-day      3641
rain                   2387
0.0                    2012
partly-cloudy-night    1462
0.0001                  993
                       ... 
0.1805                    1
0.0527                    1
0.0358                    1
0.0422                    1
0.0495                    1
Name: count, Length: 704, dtype: int64


## 4️⃣ Normalizar Nombres de Síntomas

Problema identificado: 9,000+ síntomas únicos con muchos duplicados por:
- Mayúsculas/minúsculas: "Diarrhea" vs "diarrhea"
- Espacios extra
- Sinónimos: "loose stools" vs "diarrhea"
- Typos: "Diarrhea" vs "Diarrea"

In [25]:
# Analizar síntomas
symptoms_df = df[df['trackable_type'] == 'Symptom'].copy()
print(f'Síntomas únicos (sin limpiar): {symptoms_df["trackable_name"].nunique():,}')

# Normalización básica: lowercase, strip, remove extra spaces
symptoms_df['symptom_normalized'] = (
    symptoms_df['trackable_name']
    .str.lower()
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)  # Reemplazar múltiples espacios por uno
)

print(f'Síntomas únicos (normalizado): {symptoms_df["symptom_normalized"].nunique():,}')
print(f'Reducción: {symptoms_df["trackable_name"].nunique() - symptoms_df["symptom_normalized"].nunique():,} duplicados eliminados')

# Top síntomas después de normalizar
print('\nTop 20 síntomas (normalizados):')
top_symptoms = symptoms_df['symptom_normalized'].value_counts().head(20)
print(top_symptoms)

Síntomas únicos (sin limpiar): 2,159
Síntomas únicos (normalizado): 2,152
Reducción: 7 duplicados eliminados

Top 20 síntomas (normalizados):
symptom_normalized
diarrhea                 8307
fatigue                  6696
nausea                   5372
abdominal pain           5202
headache                 4112
joint pain               3926
bloating                 3292
stomach pain             3114
constipation             2838
fatigue and tiredness    2678
bloody stools            2562
gas                      2253
bowel urgency            1932
anxiety                  1892
brain fog                1863
lower abdomen pain       1842
stiffness                1790
bowel movements          1720
depression               1454
lightheadedness          1433
Name: count, dtype: int64


In [26]:
# Identificar síntomas relacionados con Crohn/IBD
# Basado en literatura médica: https://www.crohnscolitisfoundation.org/
IBD_CORE_SYMPTOMS = {
    'diarrhea': ['diarrhea', 'diarrhoea', 'loose stools', 'loose stool', 'watery stool'],
    'abdominal_pain': ['abdominal pain', 'stomach pain', 'belly pain', 'cramping', 'cramps'],
    'blood_in_stool': ['blood in stool', 'bloody stool', 'rectal bleeding', 'bleeding'],
    'fatigue': ['fatigue', 'tired', 'tiredness', 'exhaustion', 'low energy'],
    'nausea': ['nausea', 'nauseous', 'feeling sick'],
    'fever': ['fever', 'high temperature', 'temperature'],
    'weight_loss': ['weight loss', 'losing weight'],
    'joint_pain': ['joint pain', 'arthralgia', 'joints hurt'],
}

def map_to_core_symptom(symptom_text):
    """
    Mapea síntomas a categorías core de IBD
    """
    if pd.isna(symptom_text):
        return 'other'
    
    symptom_lower = str(symptom_text).lower()
    
    for core_symptom, keywords in IBD_CORE_SYMPTOMS.items():
        for keyword in keywords:
            if keyword in symptom_lower:
                return core_symptom
    
    return 'other'

# Aplicar mapeo
symptoms_df['symptom_category'] = symptoms_df['symptom_normalized'].apply(map_to_core_symptom)

print('Distribución de categorías de síntomas:')
print(symptoms_df['symptom_category'].value_counts())

print(f'\n% de síntomas categorizados (no "other"): {(symptoms_df["symptom_category"] != "other").sum() / len(symptoms_df) * 100:.1f}%')

Distribución de categorías de síntomas:
symptom_category
other             113386
abdominal_pain     12131
fatigue            10601
diarrhea            9395
nausea              5517
blood_in_stool      4576
joint_pain          4424
fever                797
weight_loss          259
Name: count, dtype: int64

% de síntomas categorizados (no "other"): 29.6%


## 5️⃣ Limpieza de Datos Demográficos

In [27]:
# Analizar valores faltantes
print('Valores faltantes por columna:')
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False))

# Analizar edad
print('\nEdad:')
print(f'  Missing: {df["age"].isnull().sum():,} ({df["age"].isnull().sum()/len(df)*100:.1f}%)')
print(f'  Media: {df["age"].mean():.1f}')
print(f'  Mediana: {df["age"].median():.1f}')
print(f'  Rango: {df["age"].min():.0f} - {df["age"].max():.0f}')

# Edad por usuario (debería ser constante)
age_by_user = df.groupby('user_id')['age'].agg(['first', 'nunique', 'count'])
inconsistent_age = age_by_user[age_by_user['nunique'] > 1]
print(f'\nUsuarios con edad inconsistente: {len(inconsistent_age)}')

# Sexo
print('\nSexo:')
print(df['sex'].value_counts(dropna=False))

Valores faltantes por columna:
                 Missing  Percentage
value_numeric     122292   31.295536
trackable_value    46290   11.845994
value_clean        46290   11.845994
age                 7222    1.848170
country             7199    1.842284
sex                 5676    1.452535
trackable_name         1    0.000256

Edad:
  Missing: 7,222 (1.8%)
  Media: 36.4
  Mediana: 35.0
  Rango: -19963 - 78

Usuarios con edad inconsistente: 0

Sexo:
sex
female        304820
male           36177
other          22648
doesnt_say     21444
NaN             5676
Name: count, dtype: int64


## 6️⃣ Crear Dataset Limpio

Aplicar todas las transformaciones al dataframe completo

In [28]:
# Hacer una copia para el dataset limpio
df_clean = df.copy()

# 1. Normalizar trackable_name (lowercase, strip, spaces)
df_clean['trackable_name_normalized'] = (
    df_clean['trackable_name']
    .str.lower()
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
)

# 2. Crear columnas específicas por tipo
# Para síntomas: severidad 0-4
is_symptom = df_clean['trackable_type'] == 'Symptom'
df_clean.loc[is_symptom, 'symptom_severity'] = pd.to_numeric(
    df_clean.loc[is_symptom, 'trackable_value'], 
    errors='coerce'
)

# Limitar a rango 0-4 (algunos valores pueden estar fuera)
df_clean.loc[is_symptom, 'symptom_severity'] = df_clean.loc[is_symptom, 'symptom_severity'].clip(0, 4)

# Para tratamientos: extraer dosis
is_treatment = df_clean['trackable_type'] == 'Treatment'
df_clean.loc[is_treatment, 'treatment_dose'] = (
    df_clean.loc[is_treatment, 'trackable_value'].apply(extract_dose)
)

# 3. Mapear síntomas a categorías
df_clean.loc[is_symptom, 'symptom_category'] = (
    df_clean.loc[is_symptom, 'trackable_name_normalized'].apply(map_to_core_symptom)
)

# 4. Crear flag para registros completamente numéricos
df_clean['has_numeric_value'] = pd.to_numeric(df_clean['trackable_value'], errors='coerce').notna()

print('✓ Transformaciones aplicadas')
print(f'\nNuevas columnas creadas:')
new_cols = ['trackable_name_normalized', 'symptom_severity', 'treatment_dose', 'symptom_category', 'has_numeric_value']
for col in new_cols:
    if col in df_clean.columns:
        non_null = df_clean[col].notna().sum()
        print(f'  {col}: {non_null:,} valores no nulos')

✓ Transformaciones aplicadas

Nuevas columnas creadas:
  trackable_name_normalized: 390,764 valores no nulos
  symptom_severity: 161,086 valores no nulos
  treatment_dose: 65,806 valores no nulos
  symptom_category: 161,086 valores no nulos
  has_numeric_value: 390,765 valores no nulos


## 7️⃣ Filtrar Usuarios Viables para Modelado

Criterios para usuarios viables:
- Al menos 30 días de datos
- Al menos 50 registros totales
- Al menos 10 registros de síntomas

In [29]:
# Calcular métricas por usuario
user_stats = df_clean.groupby('user_id').agg({
    'checkin_date': ['min', 'max', 'nunique'],
    'trackable_type': 'count',
}).reset_index()

user_stats.columns = ['user_id', 'first_date', 'last_date', 'days_tracked', 'total_records']
user_stats['date_range_days'] = (user_stats['last_date'] - user_stats['first_date']).dt.days

# Contar registros de síntomas por usuario
symptom_counts = df_clean[df_clean['trackable_type'] == 'Symptom'].groupby('user_id').size().reset_index(name='symptom_records')
user_stats = user_stats.merge(symptom_counts, on='user_id', how='left')
user_stats['symptom_records'] = user_stats['symptom_records'].fillna(0)

print('Estadísticas de usuarios:')
print(f'  Total usuarios: {len(user_stats):,}')
print(f'  Días promedio: {user_stats["date_range_days"].mean():.1f}')
print(f'  Registros promedio: {user_stats["total_records"].mean():.1f}')
print(f'  Síntomas promedio: {user_stats["symptom_records"].mean():.1f}')

# Aplicar filtros
viable_users = user_stats[
    (user_stats['date_range_days'] >= 30) &
    (user_stats['total_records'] >= 50) &
    (user_stats['symptom_records'] >= 10)
]

print(f'\n✓ Usuarios viables: {len(viable_users):,} ({len(viable_users)/len(user_stats)*100:.1f}%)')
print(f'  Días promedio: {viable_users["date_range_days"].mean():.1f}')
print(f'  Registros promedio: {viable_users["total_records"].mean():.1f}')
print(f'  Síntomas promedio: {viable_users["symptom_records"].mean():.1f}')

Estadísticas de usuarios:
  Total usuarios: 2,046
  Días promedio: 64.8
  Registros promedio: 191.0
  Síntomas promedio: 78.7

✓ Usuarios viables: 325 (15.9%)
  Días promedio: 281.4
  Registros promedio: 1027.9
  Síntomas promedio: 431.5


In [30]:
# Filtrar dataset a usuarios viables
df_viable = df_clean[df_clean['user_id'].isin(viable_users['user_id'])].copy()

print(f'Dataset viable:')
print(f'  Registros: {len(df_viable):,} (de {len(df_clean):,})')
print(f'  Usuarios: {df_viable["user_id"].nunique():,}')
print(f'  Reducción: {(1 - len(df_viable)/len(df_clean))*100:.1f}%')

print(f'\nDistribución por tipo (usuarios viables):')
print(df_viable['trackable_type'].value_counts())

Dataset viable:
  Registros: 334,073 (de 390,765)
  Usuarios: 325
  Reducción: 14.5%

Distribución por tipo (usuarios viables):
trackable_type
Symptom      140251
Treatment     62394
Weather       50100
Condition     45938
Tag           18820
Food          16512
HBI              58
Name: count, dtype: int64


## 8️⃣ Guardar Datos Limpios

In [31]:
# Guardar dataset limpio completo
import os

output_file = '../data/processed/crohn_cleaned.csv'
df_clean.to_csv(output_file, index=False)
print(f'✓ Guardado dataset limpio: {output_file}')
print(f'  Tamaño: {os.path.getsize(output_file)/(1024**2):.2f} MB')

# Guardar dataset viable (para modelado)
viable_file = '../data/processed/crohn_viable.csv'
df_viable.to_csv(viable_file, index=False)
print(f'\n✓ Guardado dataset viable: {viable_file}')
print(f'  Tamaño: {os.path.getsize(viable_file)/(1024**2):.2f} MB')

# Guardar estadísticas de usuarios viables
user_stats_file = '../data/processed/user_statistics.csv'
viable_users.to_csv(user_stats_file, index=False)
print(f'\n✓ Guardadas estadísticas de usuarios: {user_stats_file}')

✓ Guardado dataset limpio: ../data/processed/crohn_cleaned.csv
  Tamaño: 48.79 MB

✓ Guardado dataset viable: ../data/processed/crohn_viable.csv
  Tamaño: 41.85 MB

✓ Guardadas estadísticas de usuarios: ../data/processed/user_statistics.csv


## ✅ Resumen

### Transformaciones aplicadas:
1. ✅ Normalización de nombres (lowercase, strip, espacios)
2. ✅ Separación de valores numéricos vs categóricos
3. ✅ Extracción de severidad de síntomas (0-4)
4. ✅ Extracción de dosis de tratamientos
5. ✅ Mapeo de síntomas a categorías core de IBD
6. ✅ Filtrado de usuarios viables (≥30 días, ≥50 registros, ≥10 síntomas)

### Archivos generados:
- `crohn_cleaned.csv`: Dataset completo limpio
- `crohn_viable.csv`: Solo usuarios viables para modelado
- `user_statistics.csv`: Estadísticas por usuario

### Próximos pasos:
1. **Feature Engineering** (`03_feature_engineering.ipynb`):
   - Pivotar datos a formato wide (user-date)
   - Crear rolling windows (3, 7, 14 días)
   - Calcular tendencias de síntomas
   - **Definir variable objetivo "flare"**

2. **Definir "Flare"**:
   - Opción 1: Spike en severidad promedio de síntomas
   - Opción 2: HBI score > threshold (si hay suficientes datos)
   - Opción 3: Combinación de síntomas core (diarrhea + pain + blood)